# Sensor Fault Example

In [ ]:
from IPython.display import display, HTML
display(HTML('<a target="_blank" href="https://colab.research.google.com/github/WaterFutures/EPyT-Flow/blob/main/docs/examples/sensor_fault.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>'))

This example demonstrates how add sensor faults to a scenario -- more information can be found in the [documentation](https://epyt-flow.readthedocs.io/en/stable/tut.events.html#sensor-faults).

In [ ]:
%pip install epyt-flow --quiet

In [ ]:
from epyt_flow.data.networks import load_hanoi
from epyt_flow.simulation import ScenarioSimulator, SENSOR_TYPE_NODE_PRESSURE
from epyt_flow.simulation.events import SensorFaultStuckZero
from epyt_flow.utils import to_seconds, plot_timeseries_data

Load the [Hanoi network](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.data.html#epyt_flow.data.networks.load_hanoi) with default sensor configuration:

In [ ]:
hanoi_network_config = load_hanoi(include_default_sensor_placement=True, verbose=False)

Create a new simulation:

In [ ]:
sim = ScenarioSimulator(scenario_config=hanoi_network_config)

Set simulaton duration to two days:

In [ ]:
sim.set_general_parameters(simulation_duration=to_seconds(days=2))

Add a pressure sensor fault (i.e. power failure, sensor readings are set to zero) at node "16" that is active for 90min (i.e. starts at 150min after simulation begin and ends at 240min after simulation start).

This is done by creating a new instance of the [SensorFaultStuckZero](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.events.html#epyt_flow.simulation.events.sensor_faults.SensorFaultStuckZero) class and adding this sensor fault to the scenario by calling [add_sensor_fault()](https://epyt-flow.readthedocs.io/en/stable/epyt_flow.simulation.html#epyt_flow.simulation.scenario_simulator.ScenarioSimulator.add_sensor_fault):

In [ ]:
sensor_fault = SensorFaultStuckZero(sensor_id="16",
                                    sensor_type=SENSOR_TYPE_NODE_PRESSURE,
                                    start_time=to_seconds(minutes=150),
                                    end_time=to_seconds(minutes=240))

sim.add_sensor_fault(sensor_fault)

Run the entire simulation:

In [ ]:
scada_data = sim.run_simulation()

Retrieve pressure readings at node "16":

In [ ]:
plot_timeseries_data(scada_data.get_data_pressures(sensor_locations=["16"]).T,
                     x_axis_label="Time (30min steps)", y_axis_label="Pressure in $m$")

Do not forget to close the simulator!

In [ ]:
sim.close()